In [56]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
%matplotlib tk

data = pd.read_pickle('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/xls_testruns/doe_pickle_1458.pkl') # this one for average times

In [57]:
data

,tm,dt,yy,mm,ww,wd,dd,xl,hh,an,vb,vl,ht,tt,acw,bz,dectt,decht,decacw
2016-11-14 00:00:00,00:00:00,2016-11-14,2016,11,46,Mon,14,42688,0,0,0,0,0,0,0,k,0:00:00,0:00:00,0:00:00
2016-11-14 01:00:00,01:00:00,2016-11-14,2016,11,46,Mon,14,42688,1,0,0,0,0,0,0,k,0:00:00,0:00:00,0:00:00
2016-11-14 02:00:00,02:00:00,2016-11-14,2016,11,46,Mon,14,42688,2,0,0,0,0,0,0,k,0:00:00,0:00:00,0:00:00
2016-11-14 03:00:00,03:00:00,2016-11-14,2016,11,46,Mon,14,42688,3,0,0,0,0,0,0,k,0:00:00,0:00:00,0:00:00
2016-11-14 04:00:00,04:00:00,2016-11-14,2016,11,46,Mon,14,42688,4,0,0,0,0,0,0,k,0:00:00,0:00:00,0:00:00
2016-11-14 05:00:00,05:00:00,2016-11-14,2016,11,46,Mon,14,42688,5,0,0,0,0,0,0,k,0:00:00,0:00:00,0:00:00
2016-11-14 06:00:00,06:00:00,2016-11-14,2016,11,46,Mon,14,42688,6,1,0,1,0,0,0,k,0:00:00,0:00:00,0:00:00
2016-11-14 07:00:00,07:00:00,2016-11-14,2016,11,46,Mon,14,42688,7,5,4,1,0.0244907,0.0122917,0.0121991,k,0:00:17.700000,0:00:35.266667,0:00:17.566667
2016-11-14 08:00:00,08:00:00,2016-11-14,2016,11,46,Mon,14,42688,8,46,27,19,0.110498,0.0612616,0.0492361,k,0:01:28.216667,0:02:39.116667,0:01:10.900000
2016-11-14 09:00:00,09:00:00,2016-11-14,2016,11,46,Mon,14,42688,9,74,36,38,0.159086,0.0889005,0.0701852,k,0:02:08.016667,0:03:49.083333,0:01:41.066667


In [58]:
print(data.columns)

Index(['tm', 'dt', 'yy', 'mm', 'ww', 'wd', 'dd', 'xl', 'hh', 'an', 'vb', 'vl',
       'ht', 'tt', 'acw', 'bz', 'dectt', 'decht', 'decacw'],
      dtype='object')


In [59]:
colfunx_collapse_dt={'yy':'first', 'mm':'first', 'ww':'first', 'wd':'first', 'dd':'first', 'xl':'first', 'an':'sum', 'vb':'sum', 'vl':'sum', 'ht':'sum', 'tt':'sum', 'acw':'sum','decht':'sum', 'dectt':'sum', 'decacw':'sum'}
# left out dt bz and hh; dt and bz should be indices, hh can be dropped
data_grouped=data.groupby(['dt','bz']).agg(colfunx_collapse_dt)

In [88]:
# dates without neben- or kernzeit should contain each index, but fill them with null. Solution see https://stackoverflow.com/questions/17287933/filling-in-date-gaps-in-multiindex-pandas-dataframe
dg=data_grouped.unstack('dt').fillna(0).stack('dt')
dg2=dg.reorder_levels(['dt','bz'])

In [104]:
dg2['aht']=dg2['ht']/dg2['vb']
dg2.tail()
#dg2.loc[datetime.date(2017,10,1)]


,,xl,tt,mm,decacw,an,ww,wd,dectt,vb,vl,decht,yy,ht,dd,acw,aht
dt,bz,,,,,,,,,,,,,,,,
2017-10-04,n,43012.0,0.001424,10.0,00:00:01,1.0,40.0,Wed,00:00:02.050000,1.0,0.0,00:00:03.050000,2017.0,0.002118,4.0,0.000694,0.002118
2017-10-05,n,43013.0,0.001238,10.0,00:00:01.083333,6.0,40.0,Thu,00:00:01.783333,4.0,2.0,00:00:02.866667,2017.0,0.001991,5.0,0.000752,0.000498
2017-10-06,n,43014.0,0.004873,10.0,00:00:02.250000,4.0,40.0,Fri,00:00:07.016667,3.0,1.0,00:00:09.266667,2017.0,0.006435,6.0,0.001563,0.002145
2017-10-07,n,43015.0,0.020405,10.0,00:00:10.749999,31.0,40.0,Sat,00:00:29.383334,26.0,5.0,00:00:40.133335,2017.0,0.027870,7.0,0.007465,0.001072
2017-10-08,n,43016.0,0.001898,10.0,00:00:01.800000,4.0,40.0,Sun,00:00:02.733333,2.0,2.0,00:00:04.533333,2017.0,0.003148,8.0,0.001250,0.001574


In [114]:
dg2['aht']=(dg2['ht']/dg2['vb']).fillna(0)
dg2['att']=(dg2['tt']/dg2['vb']).fillna(0)
dg2['aacw']=(dg2['acw']/dg2['vb']).fillna(0)

In [115]:
kernzeit=dg2.xs('k', level=1, drop_level=False)
nebnzeit=dg2.xs('n', level=1, drop_level=False)
# both frames share the same index (level 0, i.e. datetime.date)

In [116]:
kernzeit.tail()

,,xl,tt,mm,decacw,an,ww,wd,dectt,vb,vl,decht,yy,ht,dd,acw,aht,att,aacw
dt,bz,,,,,,,,,,,,,,,,,,
2017-10-04,k,43012.0,0.600822,10.0,00:03:00.900001,326.0,40.0,Wed,00:14:25.183332,268.0,58.0,00:17:26.083334,2017.0,0.726447,4.0,0.125625,0.002711,0.002242,0.000469
2017-10-05,k,43013.0,0.627847,10.0,00:02:40.533333,303.0,40.0,Thu,00:15:04.100001,257.0,46.0,00:17:44.633335,2017.0,0.739329,5.0,0.111481,0.002877,0.002443,0.000434
2017-10-06,k,43014.0,0.815104,10.0,00:03:45.000001,371.0,40.0,Fri,00:19:33.750001,332.0,39.0,00:23:18.750000,2017.0,0.971354,6.0,0.156250,0.002926,0.002455,0.000471
2017-10-07,k,43015.0,0.140706,10.0,00:00:38.633333,66.0,40.0,Sat,00:03:22.616668,50.0,16.0,00:04:01.249998,2017.0,0.167535,7.0,0.026829,0.003351,0.002814,0.000537
2017-10-08,k,0.0,0.000000,0.0,00:00:00,0.0,0.0,0,00:00:00,0.0,0.0,00:00:00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
